<a href="https://colab.research.google.com/github/rbajrac/Decoding-Chess-Puzzle-Difficulty/blob/main/DecodingChessPuzzleDifficulty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pickle
# Load the best-performing model from the pickle file
with open('/content/drive/MyDrive/model.pkl', 'rb') as f: #replace with correct filepath
  model = pickle.load(f)


In [ ]:
def count_captures(fen):
  #returns number of captures

    board = chess.Board(fen)
    legal_moves = list(board.legal_moves)

    checks_and_captures_count = 0

    for move in legal_moves:
        # Check if the move is a capture or results in a check
        if board.is_capture(move):
            checks_and_captures_count += 1

    return checks_and_captures_count


def calculate_material_ratio_adjusted(fen):

    piece_values = {
        'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9,
        'p': 1, 'n': 3, 'b': 3, 'r': 5, 'q': 9
    }

    total_material_side_to_move = 0
    total_material_opponent = 0

    # Get board setup and color to move from FEN string
    parts = fen.split(' ')
    board_setup = parts[0]
    active_color = parts[1]

    # Iterate over each character in the board setup
    for char in board_setup:
        if char in piece_values:
            if char.isupper():
                total_material_side_to_move += piece_values[char]
            else:
                total_material_opponent += piece_values[char]

    # If it's black's turn to move, swap the totals
    if active_color == 'b':
        total_material_side_to_move, total_material_opponent = total_material_opponent, total_material_side_to_move

    # Avoid division by zero
    if total_material_opponent == 0:
        return float('inf')
    else:
        return total_material_side_to_move / total_material_opponent

# Example usage:
fen_string = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
material_ratio = calculate_material_ratio_adjusted(fen_string)
print("Material ratio:", material_ratio)


Material ratio: 1.0


In [ ]:
import numpy as np

def fen_to_extended_grid(fen):
    # piece encoding dictionary
    piece_encoding = {
        'p': -1, 'r': -2, 'n': -3, 'b': -4, 'q': -5, 'k': -6,
        'P': 1, 'R': 2, 'N': 3, 'B': 4, 'Q': 5, 'K': 6
    }

    # initialize empty 9x8 grid
    grid = np.zeros((9, 8), dtype=int)

    # split FEN string into parts
    parts = fen.split(' ')
    rows = parts[0].split('/')

    # Fill the grid with piece encodings
    for i, row in enumerate(rows):
        col = 0
        for char in row:
            if char.isdigit():
                col += int(char)
            else:
                grid[i, col] = piece_encoding[char]
                col += 1

    # encode castling rights
    castling = parts[2]
    grid[8, 0] = 1 if 'K' in castling else 0
    grid[8, 1] = 1 if 'Q' in castling else 0
    grid[8, 2] = 1 if 'k' in castling else 0
    grid[8, 3] = 1 if 'q' in castling else 0

    # encode en-passant target square
    if parts[3] != '-':
        # Convert file letter to integer (a=1, b=2, ..., h=8)
        en_passant_col = ord(parts[3][0]) - ord('a') + 1
        grid[8, 4] = en_passant_col
    else:
        grid[8, 4] = 0

    return grid

# Example FEN string (starting position with potential en-passant)
fen = "rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq d3 0 1"
grid = fen_to_extended_grid(fen)


In [ ]:
import pandas as pd
import chess
import numpy as np

# Function to apply the moves and return a list of FEN strings
def generate_fen_sequence(fen, moves):
    board = chess.Board(fen)
    fen_sequence = [fen]

    for move_uci in moves:  # slightly different from original fx, since first move now doesn't have to be ignored
        move = chess.Move.from_uci(move_uci)
        board.push(move)
        fen_sequence.append(board.fen())

    return fen_sequence

# Apply the function to each row in the sampled_data dataframe
def apply_generate_fen_sequence(row):
    fen = row['new_fen']  # Use the new_fen column for the initial position
    moves = row['Moves'].split()  # Split the Moves column into a list of moves
    fen_sequence = generate_fen_sequence(fen, moves)
    return fen_sequence

# Apply the function to each row and create a new column with the list of FEN strings


# Display the dataframe with the new column



In [ ]:
import pandas as pd
data = pd.read_csv("drive/MyDrive/lichess_db_puzzle.csv")

In [ ]:

# inputs: initial board state (as a FEN string), list of moves played from initial position in uci

# get number of captures, number of moves in solution, material ratio (numerical features)

# generate fen_sequence


# use model to predict rating

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

def prepare_features(FEN, moves_list):
    num_captures = count_captures(FEN)
    material_ratio = calculate_material_ratio_adjusted(FEN)
    num_moves = len(moves_list.split(' '))

    fen_sequence = generate_fen_sequence(FEN, moves_list.split(' '))
    encoded_grids = [fen_to_extended_grid(fen) for fen in fen_sequence]

    # Flatten each grid into a single list per FEN to simplify padding
    flattened_grids = [grid.flatten() for grid in encoded_grids]

    # Create a DataFrame for a single sample
    data_dict = {
        'num_moves': [num_moves],
        'material_ratio': [material_ratio],
        'num_captures': [num_captures],
        'encoded_grids': [flattened_grids]
    }

    return pd.DataFrame(data_dict)

def predict_puzzle_difficulty(FEN, moves_list):
    features = prepare_features(FEN, moves_list)

    # Extract numerical features and sequence
    X_numerical = features[['num_moves', 'material_ratio', 'num_captures']].values
    X_sequence = np.array(features['encoded_grids'].iloc[0])  # Directly take the first element

    # Pad and reshape sequence
    X_sequence_padded = pad_sequences([X_sequence], maxlen=30, padding='post', truncating='post')
    X_sequence_padded = X_sequence_padded.reshape((1, -1, 9, 8, 1))  # Reshape for model input

    # Prediction
    return model.predict([X_numerical, X_sequence_padded])

# Example usage
predict_puzzle_difficulty(FEN, moves)


1/1 [==============================] - 1s 557ms/step


array([[1467.4103]], dtype=float32)

In [ ]:
FEN = '8/5R2/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 b - - 2 64'

moves = 'f5e5 e2f1 e5e6'

predict_puzzle_difficulty(FEN, moves)

1/1 [==============================] - 0s 180ms/step


array([[1467.4103]], dtype=float32)

['f5e5', 'e2f1', 'e5e6']

In [ ]:
moves = 'c3c1 g1g2 h4h3 g2g3 f6e5'
FEN = '6k1/8/1p1R1b2/1Pp2N2/2P3pp/2r1P3/5P1P/6K1 b - - 0 1'

predict_puzzle_difficulty(FEN, moves)

1/1 [==============================] - 0s 164ms/step


array([[1745.6743]], dtype=float32)

In [ ]:
FEN = 'r2qkb1r/pp1nnppp/2p1p1b1/8/3PNB2/3B1N2/PPPQ1PPP/R3K2R w KQkq - 0 1'
moves = 'e4d6'
predict_puzzle_difficulty(FEN, moves)

1/1 [==============================] - 0s 127ms/step


array([[1139.2784]], dtype=float32)

In [ ]:
FEN = 'r1b1k2r/ppq2pp1/2pbp3/8/1P1P2B1/P1N1B3/2P2PP1/R2Q1RK1 b kq - 0 1'
moves = 'd6h2 g1h1 h2g1 h1g1 c7h2'
predict_puzzle_difficulty(FEN, moves)

1/1 [==============================] - 0s 213ms/step


array([[1743.5875]], dtype=float32)

In [ ]:
FEN = '8/1p6/7p/1P6/6Pk/3K3P/8/8 b - - 0 1'
moves = 'h4h3 '